In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [37]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [38]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [39]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [40]:
# tool binding
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)

In [41]:
llm.invoke('hi')

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.037343829, 'prompt_time': 0.002436133, 'queue_time': 0.008248776, 'total_time': 0.039779962}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d607a531-0773-4427-a179-229c9300494c-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})

In [42]:
llm_with_tools = llm.bind_tools([multiply])

In [43]:
llm_with_tools.invoke("hi")

AIMessage(content="I'm happy to chat with you. What would you like to talk about or what can I help you with?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 231, 'total_tokens': 255, 'completion_time': 0.049492687, 'prompt_time': 0.013943631, 'queue_time': 0.008422542, 'total_time': 0.063436318}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2fd1782c-f826-491e-94ce-dfd6e1581f36-0', usage_metadata={'input_tokens': 231, 'output_tokens': 24, 'total_tokens': 255})

In [44]:
query = HumanMessage("Can you multiply 3 and 4?")
messages = [query]

In [45]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7ftdmx46h', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 239, 'total_tokens': 258, 'completion_time': 0.111945659, 'prompt_time': 0.014321715, 'queue_time': 0.007135604999999998, 'total_time': 0.126267374}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ded74b7e-b6be-4af8-b14f-e71bac7937f0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '7ftdmx46h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 19, 'total_tokens': 258})

In [46]:
messages.append(result)
messages

[HumanMessage(content='Can you multiply 3 and 4?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7ftdmx46h', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 239, 'total_tokens': 258, 'completion_time': 0.111945659, 'prompt_time': 0.014321715, 'queue_time': 0.007135604999999998, 'total_time': 0.126267374}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ded74b7e-b6be-4af8-b14f-e71bac7937f0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '7ftdmx46h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 19, 'total_tokens': 258})]

In [47]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 4},
 'id': '7ftdmx46h',
 'type': 'tool_call'}

In [48]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='12', name='multiply', tool_call_id='7ftdmx46h')

In [49]:
messages.append(tool_result)
messages

[HumanMessage(content='Can you multiply 3 and 4?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7ftdmx46h', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 239, 'total_tokens': 258, 'completion_time': 0.111945659, 'prompt_time': 0.014321715, 'queue_time': 0.007135604999999998, 'total_time': 0.126267374}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ded74b7e-b6be-4af8-b14f-e71bac7937f0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '7ftdmx46h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 19, 'total_tokens': 258}),
 ToolMessage(content='12', name='multiply', tool_call_id='7ftdmx46h')]

In [51]:
llm_with_tools.invoke(messages).content

'The result of the multiplication is 12.'